In [ ]:
! pip install -q transformers[sentencepiece] dataset evaluate

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.49 which is incompatible.


In [ ]:
import pandas as pd
import transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -Uqq torch

In [ ]:
!pip install -Uqq transformers[sentencepiece] transformers[torch] datasets evaluate ohmeow-blurr nbdev

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.


In [ ]:

!pip install --upgrade sqlalchemy torch torchaudio torchdata torchtext


  Obtaining dependency information for sqlalchemy from https://files.pythonhosted.org/packages/9f/94/8a20e0b867857cacf7e1269efc02aa81866071df844a2598fa34245a50df/SQLAlchemy-2.0.20-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached SQLAlchemy-2.0.20-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.4 kB)
  Using cached torch-2.0.1-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
Using cached SQLAlchemy-2.0.20-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.49
    Uninstalling SQLAlchemy-1.4.49:
      Successfully uninstalled SQLAlchemy-1.4.49
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the follow

In [ ]:

!pip install -U accelerate
!pip install -U transformers

In [ ]:
%cd /content/drive/MyDrive/Course_Recommendation_System

/content/drive/MyDrive/Course_Recommendation_System


In [ ]:
df = pd.read_csv("udemy_courses_details.csv")
df.head()
df.shape

(42906, 3)

In [ ]:
model_name = "distilroberta-base"

In [ ]:
df = df.dropna().reset_index(drop=True)

In [ ]:
import collections
from collections import Counter

In [ ]:

genre_list = []  # List to store genres
category_count = Counter()
indices_to_drop = []  # Define indices_to_drop here

for idx, labels_str in enumerate(df['Labels']):
    categories_str = labels_str.strip("[]").replace("'", "")  # Remove brackets and single quotes
    categories = categories_str.split(',')  # Split by comma to create a list
    if len(categories):
        genre_list.append(categories[0])
    else:
        indices_to_drop.append(idx)

    # Process categories for counting
    category_count.update([cat.strip() for cat in categories])

# Drop the rows based on indices_to_drop and reset index
df = df.drop(indices_to_drop).reset_index(drop=True)

print(f"Number of Genres: {len(genre_list)}")
print(genre_list)
print(f"Number of Categories: {len(category_count)}")
print(category_count)


Number of Genres: 42238
['Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'IT & Software', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business', 'Business'

In [ ]:
df['Modified_Labels'] = genre_list

In [ ]:
genre_count = df['Modified_Labels'].value_counts()
threshold = int(len(df) * 0.005)
rare_genres = [cat for cat, count in genre_count.items() if count < threshold]
len(rare_genres), rare_genres[:3]

(6, ['Personal Development', 'Photography & Video', '"Lifestyle'])

In [ ]:
rare_indice_to_drop = [idx for idx, row in df.iterrows() if row['Modified_Labels'] in rare_genres]
len(rare_indice_to_drop)

391

In [ ]:
df = df.drop(rare_indice_to_drop).reset_index(drop=True)
df.shape

(41847, 4)

In [ ]:
df['Modified_Labels'].value_counts()

IT & Software           4784
Business                4720
Design                  4697
Development             4685
Lifestyle               4679
Marketing               4650
Office Productivity     4515
Finance & Accounting    4475
Music                   4429
Teaching & Academics     213
Name: Modified_Labels, dtype: int64

In [ ]:
labels = list(set(df.Modified_Labels.to_list()))
labels_count = len(labels)
labels, labels_count


(['Teaching & Academics',
  'Marketing',
  'Office Productivity',
  'Finance & Accounting',
  'Music',
  'Business',
  'Development',
  'Design',
  'IT & Software',
  'Lifestyle'],
 10)

In [ ]:
df.describe(include='object')

,Course Title,Labels,Details,Modified_Labels
count,41847,41847,41847,41847
unique,40681,8517,39766,10
top,Music Theory,"['Office Productivity,Microsoft,Excel']",Nill,IT & Software
freq,4,973,725,4784


In [ ]:
from datasets import Dataset, DatasetDict
ds = Dataset.from_pandas(df)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def tokenizer_dsec(x):
    return tokenizer(x ['Details'], truncation=True, padding=True)


In [ ]:
tokenized_ds = ds.map(tokenizer_dsec, batched=True)

Map:   0%|          | 0/41847 [00:00<?, ? examples/s]

In [ ]:
row = tokenized_ds[0]
row['Details'], row['input_ids']

("['Use SQL to query a database,Use SQL to perform data analysis,Be comfortable putting SQL and PostgreSQL on their resume,Learn to perform GROUP BY statements,Replicate real-world situations and query reports']",
 [0,
  48759,
  34447,
  41614,
  7,
  25860,
  10,
  8503,
  6,
  34447,
  41614,
  7,
  3008,
  414,
  1966,
  6,
  9325,
  3473,
  2057,
  41614,
  8,
  1869,
  48041,
  15,
  49,
  6654,
  6,
  40103,
  7,
  3008,
  28412,
  10786,
  1997,
  6,
  47493,
  14263,
  588,
  12,
  8331,
  5458,
  8,
  25860,
  690,
  44403,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [ ]:
labels

['Teaching & Academics',
 'Marketing',
 'Office Productivity',
 'Finance & Accounting',
 'Music',
 'Business',
 'Development',
 'Design',
 'IT & Software',
 'Lifestyle']

In [ ]:
def categorizer(x):
  return {"labels": [labels.index(Modified_Labels) for Modified_Labels in x['Modified_Labels']]}

In [ ]:
categorized_ds = tokenized_ds.map(categorizer, batched= True)
categorized_ds

Map:   0%|          | 0/41847 [00:00<?, ? examples/s]

Dataset({
    features: ['Course Title', 'Labels', 'Details', 'Modified_Labels', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 41847
})

In [ ]:
row = categorized_ds[0]
row['labels']

5

In [ ]:
split_ds = categorized_ds.train_test_split(0.1, seed=42)
split_ds

DatasetDict({
    train: Dataset({
        features: ['Course Title', 'Labels', 'Details', 'Modified_Labels', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 37662
    })
    test: Dataset({
        features: ['Course Title', 'Labels', 'Details', 'Modified_Labels', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4185
    })
})

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer


In [ ]:
bs = 25
epochs = 7
lr = 3.75e-4

In [ ]:
training_args_dict = {
    "output_dir": "model",
    "learning_rate": lr,
    "warmup_ratio": 0.1,
    "lr_scheduler_type": 'cosine',
    "fp16": True,
    "evaluation_strategy": 'epoch',
    "per_device_train_batch_size": bs,
    "per_device_eval_batch_size": bs * 2,
    "num_train_epochs": epochs,
    "weight_decay": 0.01,
    "report_to": 'none'
}

args = TrainingArguments(**training_args_dict)


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = labels_count)
model

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (

In [ ]:
import evaluate
import numpy as np

def accuracy(eval_preds):
    metric = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)



In [ ]:
trainer = Trainer(model,
                  args,
                  train_dataset = split_ds['train'],
                  eval_dataset = split_ds['test'],
                  tokenizer = tokenizer,
                  compute_metrics = accuracy)

In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.053800,1.038238,0.719235
2,0.946000,0.925324,0.731661
3,0.896000,0.945037,0.745520
4,0.766400,0.935759,0.754122
5,0.658500,0.781340,0.778495
6,0.566500,0.725147,0.788053
7,0.535000,0.723249,0.789725


TrainOutput(global_step=10549, training_loss=0.780976059606532, metrics={'train_runtime': 5156.2322, 'train_samples_per_second': 51.129, 'train_steps_per_second': 2.046, 'total_flos': 3.49278925814784e+16, 'train_loss': 0.780976059606532, 'epoch': 7.0})

In [ ]:
trainer.save_model('models/udemy_course_classifier_dataset')